In [ ]:
using Combinatorics: binomial
using BenchmarkTools, Printf

function make_coeffs(a::Float64, n::Int)::Vector{ComplexF64}
    coeffs = [binomial(n, k) * (-a)^(n-k) for k in 0:n]
    return reverse(coeffs)
end

function polyval(P::Vector{ComplexF64}, x::ComplexF64)::ComplexF64
    y = zero(ComplexF64)
    for a in P
        y = y * x + a
    end
    return y
end

function ras1(P::Vector{ComplexF64}, x0::ComplexF64, x1::ComplexF64)::ComplexF64
    return (polyval(P, x1) - polyval(P, x0)) / (x1 - x0)
end

function ras2(P::Vector{ComplexF64}, x0::ComplexF64, x1::ComplexF64, x2::ComplexF64)::ComplexF64
    return (ras1(P, x1, x2) - ras1(P, x0, x1)) / (x2 - x0)
end

function muller_step(P::Vector{ComplexF64}, x0::ComplexF64, x1::ComplexF64, x2::ComplexF64)::ComplexF64
    f2 = polyval(P, x2)
    d1 = ras1(P, x1, x2)
    d2 = ras1(P, x0, x2)
    A  = ras2(P, x0, x1, x2)
    B  = d1 + d2 - ras1(P, x0, x1)
    D  = sqrt(B*B - 4*A*f2)
    denom = abs(B + D) > abs(B - D) ? B + D : B - D
    return x2 - 2 * f2 / denom
end

function find_root_muller(P::Vector{ComplexF64}, x_init::ComplexF64, eps::Float64=1e-10, maxiter::Int=10000)::ComplexF64
    x0 = 0.85 * x_init
    x1 = x_init
    x2 = 1.15 * x_init
    for _ in 1:maxiter
        x3 = muller_step(P, x0, x1, x2)
        if abs(polyval(P, x3)) < eps
            return x3
        end
        x0, x1, x2 = x1, x2, x3
    end
    return x2
end

function deflate(P::Vector{ComplexF64}, root::ComplexF64)::Vector{ComplexF64}
    n = length(P)
    B = Vector{ComplexF64}(undef, n-1)
    B[1] = P[1]
    for i in 2:n-1
        B[i] = B[i-1] * root + P[i]
    end
    return B
end

function mullers_all_roots(P::Vector{ComplexF64}, eps::Float64=1e-10)::Vector{ComplexF64}
    roots = ComplexF64[]
    while length(P) > 1
        r = find_root_muller(P, 1.0 + 0.0im, eps)
        push!(roots, r)
        P = deflate(P, r)
    end
    return roots
end

P = make_coeffs(1.1, 5)
println("\nJulia Benchmark:")
P
roots = mullers_all_roots(P)


Julia Benchmark:


5-element Vector{ComplexF64}:
 1.1002997445611942 + 0.008889576064149394im
 1.0916384745571326 + 0.003032326416413989im
 1.1085473748873975 + 0.002462125503724224im
 1.1049828005257747 - 0.007368724518387231im
 1.0945316054685006 - 0.007015303465900376im

### Модифицированный метод касательных

In [ ]:

const NEWTON_TOL       = 1e-8
const NEWTON_MAX_STEPS = 1000
const UNIQUE_TOL       = 1e-8

function poly_value(z::Complex{Float64}, coeffs::Vector{Complex{Float64}})
    result = 0 + 0im
    for coeff in coeffs
        result = result * z + coeff
    end
    return result
end

function poly_derivative(z::Complex{Float64}, coeffs::Vector{Complex{Float64}})
    result = 0 + 0im
    degree = length(coeffs) - 1
    for i in 1:(length(coeffs) - 1)
        coeff = coeffs[i]
        result = result * z + coeff * (degree - (i - 1))
    end
    return result
end
function deflate(coeffs::Vector{Complex{Float64}},
                 root::Complex{Float64})
    n = length(coeffs) - 1
    new_coeffs = Vector{Complex{Float64}}(undef, n)
    new_coeffs[1] = coeffs[1]
    for k in 2:n
        new_coeffs[k] = coeffs[k] + new_coeffs[k-1] * root
    end
    return new_coeffs
end

function find_roots(coeffs_real::Vector{Float64})
    coeffs = Complex{Float64}.(coeffs_real)

    roots = Complex{Float64}[]

    while length(coeffs) > 1               

        n = length(coeffs) - 1             
        max_tail = maximum(abs.(coeffs[2:end]))
        R = 1 / max_tail / abs(coeffs[1])
        points = Vector{Complex{Float64}}(undef, n)
        for k in 1:n
            angle = 2π * (k - 1) / n
            points[k] = R * (cos(angle) + sin(angle)*im)
        end

        found_new_root = false
        for z0 in points

            z = z0
            for step in 1:NEWTON_MAX_STEPS
                f = poly_value(z, coeffs)
                df = poly_derivative(z, coeffs)
                if abs(df) < eps()
                    break
                end
                z_next = z - f / df
                if abs(z_next - z) < NEWTON_TOL
                    z = z_next
                    break
                end
                z = z_next
            end

            if !isfinite(real(z)) || !isfinite(imag(z))
                continue 
            end
            too_close = false
            for r in roots
                if abs(z - r) < UNIQUE_TOL
                    too_close = true
                    break
                end
            end
            if too_close
                continue
            end
            push!(roots, z)
            coeffs = deflate(coeffs, z)
            found_new_root = true
            break
        end

        if !found_new_root
            error("Не удалось найти новый корень - попробуйте увеличить R или кол-во итераций")
        end
    end

    return roots
end


coeffs = [1.00000000000000 -808.710000000000 246641.841200000 -33898682.5068420 1842011080.63283 -11822825758.0076 34111432268.5231 -55344083765.8971 54125863885.7217 -31826936635.5868 10407310057.2455 -1459118284.25595]

roots = find_roots(coeffs)

println("Найдено корней: ", length(roots))
for (i, r) in enumerate(roots)
    println("root #", i, ": ", real(r), " + ", imag(r), "i")
end

Найдено корней: 11
root #1: 0.9099999484803867 + 0.0i
root #2: 0.9300003036177352 + 0.0i
root #3: 0.9499994807910818 + 0.0i
root #4: 1.0000018059957385 + 0.0i
root #5: 1.0199958613623579 + 0.0i
root #6: 1.030002714204663 + 0.0i
root #7: 1.0699998855479969 + 0.0i
root #8: 200.00002052655634 + 0.0i
root #9: 200.29992668488978 + 0.0i
root #10: 200.50006159777647 + 0.0i
root #11: 200.99999119077748 + 0.0i


### Апроксимация числа $\pi$

In [ ]:
function chudnovsky_iters_simple(digits::Int; extra::Int = 40)
    oldbits = precision(BigFloat)
    newbits = ceil(Int, (digits + extra) * log(10) / log(2))
    setprecision(newbits)

    target = BigFloat(10)^(-digits)
    C      = big"640320"
    C3     = C^3
    s      = zero(BigFloat)
    pi_ = zero(BigFloat)

    k = 0
    elapsed = @elapsed begin
        while true
            num = (-1)^k * factorial(big(6k)) * (13591409 + 545140134*k)
            den = factorial(big(3k)) * factorial(big(k))^3 *
                  sqrt(C3) * C3^k
            s += num / den

            pi_ = 1 / (12 * s)
            abs(pi_- big(pi)) < target && break
            k += 1
        end
    end

    setprecision(oldbits)
    return k + 1, elapsed, pi_
end


chudnovsky_iters_simple (generic function with 1 method)

In [ ]:
digits_list = [10, 50, 100, 200, 500, 1000,2000]

@printf("%7s | %9s | %10s\n", "digits", "iters", "time, s")
println("-"^32)

for d in digits_list
    iters, elapsed, _ = chudnovsky_iters_simple(d)
    @printf("%7d | %9d | %10.6f\n", d, iters, elapsed)
end


 digits |     iters |    time, s
--------------------------------
     10 |         1 |   0.000015
     50 |         4 |   0.000024
    100 |         8 |   0.000046
    200 |        15 |   0.000096
    500 |        36 |   0.000328
   1000 |        71 |   0.000915
   2000 |       142 |   0.010019


In [ ]:
using Printf
using BenchmarkTools

function machin_pi(digits::Int)
    old = precision(BigFloat)
    setprecision(digits + 4)
    val = 4 * (4*atan(big(1)/5) - atan(big(1)/239))
    setprecision(old) 
    return val
end


function avg_time(f::Function; repeats::Int = 10)
    total_ns = 0
    for _ in 1:repeats
        t0 = time_ns()
        f()
        total_ns += (time_ns() - t0)
    end
    return (total_ns / repeats) / 1e9
end

digits_list = [50, 100, 200, 500, 1000,2000, 5000]
repeats     = 10

@printf("%7s | %14s\n", "цифр", "среднее время, s")
println("-"^26)

for d in digits_list
    t = avg_time(() -> machin_pi(d); repeats = repeats)
    @printf("%7d | %14.6f\n", d, t)
end

   цифр | среднее время, s
--------------------------
     50 |       0.000015
    100 |       0.000016
    200 |       0.000025
    500 |       0.000045
   1000 |       0.000088
   2000 |       0.000132
   5000 |       0.000472
